# Purpose:
This notebook generates a data file that indicates the closest healthcare service point for each healthcare demand area. It

In [1]:
import geopy.distance
import pandas as pd
import numpy as np
import os
coords_1 = (52.2296756, 21.0122287)
coords_2 = (52.406374, 16.9251681)

# 
print(geopy.distance.geodesic(coords_1, coords_2).km)

279.35290160430094


In [2]:
os.getcwd()

'c:\\Users\\future room dell\\GIT_REPOS\\SafeGit\\PHCPMP\\Madelaine_Gerald\\scratch_G'

In [11]:
Suppliers = pd.read_excel('../V3_Database.xlsx', sheet_name='SupplyLocations').dropna()
Suppliers["nodeID"] = np.arange(len(Suppliers))
Suppliers['Longitude'] = Suppliers['Longitude'].astype(float)


Demands = pd.read_excel('../V3_Database.xlsx').dropna()
Demands["nodeID"] = np.arange(len(Demands))

Demands['Latitud'] = Demands['Latitud'].astype(float)
Demands['Longitud'] = Demands['Longitud'].astype(float)

Suppliers['Latitude'] = Suppliers['Latitude'].astype(float)
Suppliers['Longitude'] = Suppliers['Longitude'].astype(float)

Demands.to_excel("../data/Demand_Node_Data.xlsx", index=False)
Suppliers.to_excel("../data/Supply_Node_Data.xlsx", index=False)

In [4]:
import pandas as pd

# Assuming df is your DataFrame and 'column_name' is the column you want to convert


In [5]:
display(Demands.head())

display(Suppliers.head())

,Unnamed: 0,Nombre de Corregimiento,NOMBRE DE BARRIADA,Máx. de C2,Latitud,Longitud,Update Estimated Demand,nodeID
0,1,24 DE DICIEMBRE,LOS CÁNTAROS,13137,9.088837,-79.344104,47739.105,0
1,2,ACHIOTE,(en blanco),598,9.222575,-80.019661,559.986,1
2,3,AGUA BUENA,(en blanco),1079,7.835982,-80.394666,752.220,2
3,4,AGUA DE SALUD,(en blanco),427,8.496377,-81.430353,1435.188,3
4,5,AGUA FRÍA,(en blanco),905,8.422596,-80.341990,1886.520,4


,CODIGO,REGION DE SALUD,Nivel de Atención,DENOMINACION,DISTRITO,CORREGIMIENTO,LUGAR POBLADO,EXPEDIENTE\nELECTRÓNICO,DEP.,POB. DE\nRESP.,Latitude,Longitude,nodeID
1,101010501,BOCAS DEL TORO,Hospital,HOSPITAL GUILLERMO SANCHEZ BORBON,BOCAS DEL TORO,BOCAS DEL TORO,BOCAS DEL TORO,Si tiene,MINSA,17057,9.340233,-82.242332,0
2,102010401,BOCAS DEL TORO,Hospital,HOSPITAL DE CHANGUINOLA,CHANGUINOLA,CHANGUINOLA,CHANGUINOLA,No tiene,C.S.S.,147571,9.065358,-79.497925,1
3,103012501,BOCAS DEL TORO,Hospital,HOSPITAL DE CHIRIQUÍ GRANDE,CHIRIQUI GRANDE,CHIRIQUÍ GRANDE,CHIRIQUÍ GRANDE,No tiene,C.S.S.,12946,8.957907,-82.141986,2
4,104012501,BOCAS DEL TORO,Hospital,HOSPITAL DE ALMIRANTE,ALMIRANTE,ALMIRANTE,ALMIRANTE,No tiene,C.S.S.,17677,9.306589,-82.407399,3
6,101020801,BOCAS DEL TORO,Centro de Salud Basico,C. DE S. BASTIMENTOS,BOCAS DEL TORO,BASTIMENTOS,BASTIMENTOS,Si tiene,MINSA,2251,9.302953,-82.149412,4


In [6]:
Demands.describe()

,Unnamed: 0,Máx. de C2,Latitud,Longitud,Update Estimated Demand,nodeID
count,636.000000,636.000000,636.000000,636.000000,636.000000,636.000000
mean,319.099057,1427.281447,8.504964,-80.600858,3505.992325,317.500000
std,184.523966,2367.352566,0.614585,1.322147,6761.976388,183.741666
min,1.000000,42.000000,3.311479,-82.863617,0.000000,0.000000
25%,159.750000,343.000000,8.149188,-81.583898,603.268500,158.750000
50%,318.500000,710.000000,8.496044,-80.548846,1297.281000,317.500000
75%,478.250000,1390.500000,8.969651,-79.738505,2889.330750,476.250000
max,640.000000,17973.000000,9.630601,-71.602479,53348.517000,635.000000


In [7]:
Suppliers.describe()

,CODIGO,Latitude,Longitude,nodeID
count,6.510000e+02,651.000000,651.000000,651.000000
mean,6.517449e+08,8.603917,-80.030948,325.000000
std,3.973997e+08,0.901073,7.151561,188.071795
min,1.010105e+08,0.000000,-82.863617,0.000000
25%,3.030111e+08,8.352892,-81.772581,162.500000
50%,6.030812e+08,8.650915,-80.438011,325.000000
75%,1.001011e+09,9.076379,-79.709976,487.500000
max,1.309091e+09,9.630601,0.000000,650.000000


In [9]:
# display(Demands.head())
# display(Suppliers.head())



def get_closest_pairs(dems, sups, ):
    # [dem_node, sup_node]
    closest_pairs = []
    to_drop = []
    paired_suppliers = []
    unpaired_suppliers = sups["nodeID"].tolist()
    for i in dems["nodeID"]:
        closest_pair = None
        closest_distance = np.inf
        demand = dems.loc[dems['nodeID'] == i, "Update Estimated Demand"].values.tolist()[0]
        dem_lat, dem_lon = dems.loc[dems['nodeID'] == i, 'Latitud'].values.tolist()[0], dems.loc[dems['nodeID'] == i, 'Longitud'].values.tolist()[0]
        # paired_suppliers = list(set(paired_suppliers))
        for j in sups["nodeID"]:
            # print(f"dems:\n({dem_lat},{dem_lon})")
            sup_lat, sup_lon = sups.loc[sups['nodeID'] == j, 'Latitude'].values.tolist()[0], sups.loc[sups['nodeID'] == j, 'Longitude'].values.tolist()[0]
            distance = geopy.distance.geodesic((dem_lat, dem_lon), (sup_lat, sup_lon)).km
            if distance < closest_distance:
                closest_distance = distance
                closest_pair = (i, j, distance, demand)
        # paired_suppliers.append(closest_pair[1])
        if closest_pair[1] in unpaired_suppliers:
            unpaired_suppliers.remove(closest_pair[1])
        closest_pairs.append(closest_pair)
    return closest_pairs, unpaired_suppliers




dist_pairs, unpaired_suppliers = get_closest_pairs(Demands, Suppliers)


In [ ]:
unpaired_suppliers

[7,
 9,
 10,
 11,
 12,
 16,
 17,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 54,
 63,
 64,
 66,
 69,
 74,
 76,
 78,
 80,
 81,
 82,
 83,
 84,
 85,
 87,
 88,
 89,
 90,
 92,
 93,
 94,
 95,
 98,
 101,
 102,
 105,
 107,
 108,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 119,
 120,
 121,
 123,
 126,
 128,
 138,
 142,
 143,
 144,
 145,
 147,
 149,
 151,
 154,
 156,
 157,
 158,
 160,
 161,
 162,
 164,
 166,
 168,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 180,
 181,
 182,
 183,
 185,
 186,
 188,
 190,
 191,
 199,
 203,
 207,
 212,
 217,
 219,
 230,
 239,
 240,
 242,
 245,
 255,
 257,
 261,
 262,
 263,
 267,
 268,
 269,
 271,
 273,
 275,
 276,
 282,
 283,
 284,
 288,
 289,
 291,
 293,
 294,
 297,
 300,
 312,
 316,
 317,
 327,
 328,
 329,
 330,
 339,
 349,
 350,
 357,
 359,
 361,
 362,
 367,
 369,
 370,
 371,
 374,
 390,
 393,
 394,
 401,
 410,
 411,
 412,
 413,
 414,
 415,
 417,
 419,
 420,
 421,
 422,
 423,

In [ ]:
dist_pairs

[(0, 404, 0.0, 47739.104999999996),
 (1, 165, 0.0, 559.986),
 (2, 347, 3.325127359349233, 752.2199999999999),
 (3, 610, 0.0, 1435.1879999999999),
 (4, 59, 8.807507571124841, 1886.52),
 (5, 43, 0.0, 5716.871999999999),
 (6, 544, 0.0, 6513.866999999999),
 (7, 205, 0.0, 1902.042),
 (8, 507, 0.0, 28044.672),
 (9, 3, 0.0, 2916.942),
 (10, 534, 13.72165394823804, 807.144),
 (11, 225, 5.574565501773155, 4842.267),
 (12, 574, 0.0, 1938.4589999999998),
 (13, 611, 0.0, 549.24),
 (14, 391, 0.0, 297.306),
 (15, 470, 0.0, 1992.1889999999999),
 (16, 513, 0.0, 17437.176),
 (17, 498, 0.0, 22222.728),
 (18, 59, 0.0, 6394.467),
 (19, 238, 30.108286554848497, 382.08),
 (20, 516, 0.0, 18765.501),
 (21, 471, 0.0, 349.842),
 (22, 442, 0.0, 26463.218999999997),
 (23, 213, 0.0, 3590.955),
 (24, 525, 0.0, 4633.914),
 (25, 208, 0.0, 4780.776),
 (26, 264, 4.237209082872939, 3052.461),
 (27, 626, 0.0, 2522.922),
 (28, 383, 0.0, 409.542),
 (29, 210, 0.0, 2509.191),
 (30, 5, 8.78093465789904, 1106.241),
 (31, 388, 

In [14]:
# python -m ipykernel install --user --name=ThePy3
def convert_dist_pairs_df(dist_pairs):
    dist_pairs_df = pd.DataFrame(dist_pairs, columns=["Demand_node", "Supplier_node", "Distance", "Demand"])
    dist_pairs_df = dist_pairs_df.sort_values(by=["Distance", "Demand"], ascending=[True, False])
    return dist_pairs_df
dist_pairs_df = convert_dist_pairs_df(dist_pairs)

display(dist_pairs_df)

dist_pairs_df.to_excel("../data/Distance_Pairs.xlsx", index=False)

,Demand_node,Supplier_node,Distance,Demand
599,599,499,0.000000,53348.517
0,0,404,0.000000,47739.105
441,441,400,0.000000,41958.951
290,290,428,0.000000,39684.978
548,548,495,0.000000,36590.130
...,...,...,...,...
154,154,307,34.192546,710.430
87,87,303,496.386764,0.000
90,90,303,507.318053,419.691
194,194,303,519.941561,762.369


In [ ]:
dist_pairs_df.loc[dist_pairs_df["Demand"] == 0, ["Demand"]].values.flatten().tolist()

[0.0]

In [ ]:
num_utilized_suppliers = len(pd.unique(dist_pairs_df["Supplier_node"]))

original_suppliers = Suppliers.shape[0]

print("Out of {} potential PHC facilities, {} are shown to be the closest to the demand locations.".format(original_suppliers, num_utilized_suppliers))

print("Thus {} PHC facilities are not utilized.".format(original_suppliers - num_utilized_suppliers))

Out of 651 potential PHC facilities, 396 are shown to be the closest to the demand locations.
Thus 255 PHC facilities are not utilized.
